# **For Pdf File**

In [ ]:
# Install the required libraries
!pip install opencv-python matplotlib pyzbar
!apt-get install -y libzbar0
!pip install pdf2image
!apt-get install poppler-utils
!pip install pdf2image
!apt-get install poppler-utils
!pip install easyocr

: 

In [ ]:
!pip install easyocr
import numpy as np
import itertools
import cv2
from PIL import Image
import easyocr
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from google.colab.patches import cv2_imshow
import pandas as pd
import torch

import pandas as pd
import os
from pdf2image import convert_from_path
# Import libraries
import cv2
from pyzbar.pyzbar import decode
import matplotlib.pyplot as plt


# Initialize OCR readers
easyocr_reader = easyocr.Reader(['en'])
trocr_processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
trocr_model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')
def process_img(image_path):
  # Load the image
  # image_path = '/content/3.jpg'  # Update with the correct path
  image = cv2.imread(image_path)

  # Check if the image was loaded correctly
  if image is None:
      print("Error: Could not load image. Please check the file path.")
  else:
      # Convert to grayscale
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      # Apply GaussianBlur to reduce noise and improve contour detection
      blurred = cv2.GaussianBlur(gray, (5, 5), 0)

      # Apply thresholding to get a binary image
      _, thresh = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY_INV)

      # Find contours
      contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      # Create a copy of the original image to draw on
      annotated_image = image.copy()

      squares = []

      # Loop through the contours to detect squares with size constraints
      for cnt in contours:
          x, y, w, h = cv2.boundingRect(cnt)

          # Check if the contour is approximately square and lies between 30 and 80 pixels in size
          if abs(w - h) < 10 and 30 <= w <= 80 and 30 <= h <= 80:
              # Store square data (x, y, w, h)
              squares.append((x, y, w, h))

      # Initialize ocr_results and blocks_info
      ocr_results = []
      blocks_info = []

      # Function to perform OCR on each square
      def ocr_squares(squares, image):
          results = []
          for x, y, w, h in squares:
              square_img = image[y:y+h, x:x+w]
              pil_img = Image.fromarray(cv2.cvtColor(square_img, cv2.COLOR_BGR2RGB))

              # Perform EasyOCR
              easyocr_result = easyocr_reader.readtext(np.array(pil_img))
              easyocr_text = ' '.join([res[1] for res in easyocr_result])

              # If EasyOCR fails, fall back to TrOCR
              if not easyocr_text:
                  pixel_values = trocr_processor(pil_img, return_tensors="pt").pixel_values
                  with torch.no_grad():
                      generated_ids = trocr_model.generate(pixel_values)
                  easyocr_text = trocr_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

              # Draw detected text on the image in blue color
              cv2.putText(annotated_image, easyocr_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

              results.append({
                  'x': x,
                  'y': y,
                  'w': w,
                  'h': h,
                  'detected_text': easyocr_text
              })
          return results

      # Function to detect and label blocks in a given region
      def label_blocks(squares, y_min_limit, y_max_limit, start_label, block_config=None, color=(0, 165, 255), max_blocks=None):
          blocks = []

          # Filter squares based on y-coordinate range
          region_squares = [sq for sq in squares if y_min_limit <= sq[1] <= y_max_limit]

          if block_config is None:
              # For regions without specific block size constraints, detect blocks of close squares
              max_gap = 250  # Adjust this value to allow more space between squares in the same block
              for square1, square2 in itertools.combinations(region_squares, 2):
                  x1, y1, w1, h1 = square1
                  x2, y2, w2, h2 = square2

                  # Check if squares are in the same row and close to each other within the max_gap
                  if abs(y1 - y2) < 10 and 0 < abs(x1 - (x2 + w2)) <= max_gap:
                      blocks.append([square1, square2])

              # Combine blocks if they share squares
              merged_blocks = []
              for block in blocks:
                  merged = False
                  for merged_block in merged_blocks:
                      if any(square in merged_block for square in block):
                          merged_block.extend(square for square in block if square not in merged_block)
                          merged = True
                          break
                  if not merged:
                      merged_blocks.append(block)

              # Include single squares that are not part of a block
              used_squares = {sq for block in merged_blocks for sq in block}
              single_squares = [sq for sq in region_squares if sq not in used_squares]

              # Add single squares as individual blocks
              for sq in single_squares:
                  merged_blocks.append([sq])

              # Sort blocks by x-coordinate, then by y-coordinate within each column
              sorted_blocks = sorted(merged_blocks, key=lambda b: (min(x for x, y, w, h in b), min(y for x, y, w, h in b)))
          else:
              # For region 2, first detect blocks of 2 squares
              max_gap = 250
              sorted_squares = sorted(region_squares, key=lambda sq: (sq[1], sq[0]))

              current_block = []
              for i, square in enumerate(sorted_squares):
                  if not current_block:
                      current_block.append(square)
                  else:
                      last_square = current_block[-1]
                      if abs(square[0] - (last_square[0] + last_square[2])) <= max_gap and abs(square[1] - last_square[1]) < 10:
                          current_block.append(square)
                      else:
                          if len(current_block) == 2:
                              blocks.append(current_block)
                          current_block = [square]
              if len(current_block) == 2:
                  blocks.append(current_block)

              # Include single squares that are not part of a block of 2
              used_squares = {sq for block in blocks for sq in block}
              single_squares = [sq for sq in sorted_squares if sq not in used_squares]

              # Add single squares as individual blocks
              for sq in single_squares:
                  blocks.append([sq])

              # Sort blocks by x-coordinate, then by y-coordinate within each column
              sorted_blocks = sorted(blocks, key=lambda b: (min(x for x, y, w, h in b), min(y for x, y, w, h in b)))

          # Label the blocks
          label_counter = start_label
          for block in sorted_blocks:
              if max_blocks and label_counter > max_blocks:
                  break

              x_min = min(x for x, y, w, h in block)
              y_min = min(y for x, y, w, h in block)
              x_max = max(x + w for x, y, w, h in block)
              y_max = max(y + h for x, y, w, h in block)

              # Draw the block rectangle in specified color
              cv2.rectangle(annotated_image, (x_min, y_min), (x_max, y_max), color, 2)

              # Place the block number label on the left side of the block
              label_position = (x_min - 50, y_min + (y_max - y_min) // 2)  # Adjust position
              cv2.putText(annotated_image, str(label_counter), label_position, cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 2)

              # Perform OCR on the blocks
              block_squares = [sq for sq in squares if x_min <= sq[0] <= x_max and y_min <= sq[1] <= y_max]
              block_ocr_results = ocr_squares(block_squares, image)

              # Collect block information
              blocks_info.append({
                  'Block no': label_counter,
                  'no of squares in block': len(block_squares),
                  'Text in that block': ', '.join(result['detected_text'] for result in block_ocr_results)
              })

              # Increment the label counter
              label_counter += 1

          return label_counter


      # Function to label blocks specifically for Region 3
      def label_blocks_region_3(squares, y_min_limit, y_max_limit, start_label):
          blocks = []
          # Filter squares based on y-coordinate range
          region_squares = [sq for sq in squares if y_min_limit <= sq[1] <= y_max_limit]

          # Sort squares by y-coordinate first, then by x-coordinate to group by rows
          sorted_squares = sorted(region_squares, key=lambda sq: (sq[1], sq[0]))

          current_block = []
          last_y = None

          for square in sorted_squares:
              x, y, w, h = square
              if last_y is None or abs(y - last_y) < 10:
                  current_block.append(square)
              else:
                  if current_block:
                      blocks.append(current_block)
                  current_block = [square]
              last_y = y

          if current_block:
              blocks.append(current_block)

          # Label the blocks
          label_counter = start_label
          for block in blocks:
              x_min = min(x for x, y, w, h in block)
              y_min = min(y for x, y, w, h in block)
              x_max = max(x + w for x, y, w, h in block)
              y_max = max(y + h for x, y, w, h in block)

              # Skip blocks that contain only one square with text 'n'
              if len(block) == 1:
                  block_squares = block
                  block_ocr_results = ocr_squares(block_squares, image)
                  text_in_block = ', '.join(result['detected_text'] for result in block_ocr_results)
                  if text_in_block == 'n':
                      continue

              # Draw the block rectangle in green color
              cv2.rectangle(annotated_image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green color (BGR)

              # Place the block number label on the left side of the block
              label_position = (x_min - 50, y_min + (y_max - y_min) // 2)  # Adjust position
              cv2.putText(annotated_image, str(label_counter), label_position, cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

              # Perform OCR on the blocks
              block_squares = [sq for sq in squares if x_min <= sq[0] <= x_max and y_min <= sq[1] <= y_max]
              block_ocr_results = ocr_squares(block_squares, image)

              # Collect block information
              blocks_info.append({
                  'Block no': label_counter,
                  'no of squares in block': len(block_squares),
                  'Text in that block': ', '.join(result['detected_text'] for result in block_ocr_results)
              })

              # Increment the label counter
              label_counter += 1

          return label_counter


      # Define the function to detect and decode barcodes
      def detect_and_decode_barcode(image):
          gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          barcodes = decode(gray)

          if not barcodes:
            print("No barcodes detected.")
            return None
          for barcode in barcodes:
              barcode_data = barcode.data.decode("utf-8")
              barcode_type = barcode.type
              (x, y, w, h) = barcode.rect
              cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
              cv2.putText(image, f"{barcode_data} ({barcode_type})",
                          (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
          image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          return barcode_data



      # Detect barcode in the image
      barcode_data = detect_and_decode_barcode(image)

      # Initialize blocks_info and add the first row for Block 1
      blocks_info = []

      if barcode_data:
          # If barcode data is detected, add it as Block 1
          blocks_info.append({
              'Block no': 1,
              'no of squares in block': 0,  # Barcode detection is treated as a separate block
              'Text in that block': barcode_data,  # Add the detected barcode data
              'Label': barcode_data  # Use barcode data as the label
          })
      else:
          # If no barcode is detected, still add Block 1 with an empty label
          blocks_info.append({
              'Block no': 1,
              'no of squares in block': 0,
              'Text in that block': 'None',
              'Label': 'None'  # Leave the label empty if no barcode is detected
          })


      # Label blocks for Region 1 (0 to 1050) with blue color, limit to blocks 2 to 11
      next_label = label_blocks(squares, 0, 1050, start_label=2, color=(255, 0, 0), max_blocks=11)

      # Label blocks for Region 2 (1050 to 2260) with green color
      next_label = label_blocks(squares, 1050, 2260, start_label=next_label, block_config={'block_size': 2}, color=(0, 255, 0))

      # Label blocks for Region 3 (2260 to end) with red color
      next_label = label_blocks_region_3(squares, 2260, float('inf'), start_label=next_label)

      # Create a DataFrame from blocks_info
      df_blocks = pd.DataFrame(blocks_info)

      # Save the blocks info to a CSV file
      blocks_info_csv_path = '/content/blocks_info.csv'
      # blocks_info_csv_pathh = '/content/updated_blocks_info.csv'
      df_blocks.to_csv(blocks_info_csv_path, index=False)
      # df_blocks.to_csv(blocks_info_csv_pathh, index=False)

      # Load the blocks info CSV
      blocks_info_df = pd.read_csv(blocks_info_csv_path)

      # Define a function to apply the labeling logic
      def apply_labeling(row):
          num_squares = row['no of squares in block']
          text_in_block = row['Text in that block']

          if num_squares == 1:
            # Check if the text_in_block is 'S' or 'N'
            if text_in_block in {'S', 'N'}:
                return text_in_block
            else:
                return 'Not Marked'

          elif num_squares == 2:
              # Split the text and check for 'S' and 'N'
              texts = set(text_in_block.split(', '))
              valid_texts = {'S', 'N'}

              # If any text other than 'S' or 'N' is detected, clear the Text in that block
              if not texts.issubset(valid_texts):
                  row['Text in that block'] = ''
                  detected_labels = texts.intersection(valid_texts)
                  if len(detected_labels) == 1:
                      # If only one of 'S' or 'N' is detected, return it as the label
                      return detected_labels.pop()
                  else:
                      # If both 'S' and 'N' are detected, return 'Not Marked'
                      return 'Not Marked'
              else:
                  # If both 'S' and 'N' are detected, return 'Not Marked'
                  if 'S' in texts and 'N' in texts:
                      return 'Not Marked'

          elif num_squares == 5:
              # Split the text and check for digits 1-6
              texts = set(text_in_block.split(', '))
              valid_digits = {'1', '2', '3', '4', '5', '6'}

              # Determine which digits are missing
              missing_digits = valid_digits - texts

              # If more than one digit is missing
              if len(missing_digits) > 1:
                  return 'Not Marked'
              elif len(missing_digits) == 1:
                  return f" {missing_digits.pop()}"
          elif num_squares == 6:
              # Split the text and check for digits 1-6
              texts = set(text_in_block.split(', '))
              valid_digits = {'1', '2', '3', '4', '5', '6'}

              # Determine which digits are missing
              missing_digits = valid_digits - texts

              if len(missing_digits) == 0:
                  # All digits from 1 to 6 are present
                  return 'Not Marked'
              elif len(missing_digits) > 1:
                  # More than one digit is missing
                  return 'Not Marked'
              else:
                  # Only one digit is missing
                  return f"{missing_digits.pop()}"

          elif num_squares == 4:
              # Split the text and check for 'S'
              texts = text_in_block.split(', ')

              # Count occurrences of 'S'
              s_count = texts.count('S')

              if s_count == 4:
                  # All entries are 'S'
                  return 'Not Marked'
              elif s_count == 3:
                  # One 'S' is missing, find which square is missing 'S'
                  missing_index = [i+1 for i, text in enumerate(texts) if text != 'S']
                  if len(missing_index) == 1:
                      return f"{2025 - missing_index[0]}"  # Adjust calculation if needed
                  else:
                      return 'Not Marked'
              else:
                  # Check for 'N' if no or fewer 'S' are found
                  n_count = texts.count('N')
                  if n_count > 0:
                      return 'N'
                  return 'Not Marked'

          elif num_squares == 3:
              # Split the text and check for 'S'
              texts = text_in_block.split(', ')

              # Count occurrences of 'S'
              s_count = texts.count('S')

              if s_count == 4:
                  # All entries are 'S'
                  return 'Not Marked'
              elif s_count == 3:
                  # One 'S' is missing, find which square is missing 'S'
                  missing_index = [i+1 for i, text in enumerate(texts) if text != 'S']
                  if len(missing_index) == 1:
                      return f"{2025 - missing_index[0]}"  # Adjust calculation if needed
                  else:
                      return 'Not Marked'
              else:
                  # Check for 'N' if no or fewer 'S' are found
                  n_count = texts.count('N')
                  if n_count > 0:
                      return 'N'
                  return 'Not Marked'

          # Default case: keep existing label or leave empty
          return ''

      # Apply the labeling function to the DataFrame
      blocks_info_df['Label'] = blocks_info_df.apply(apply_labeling, axis=1)

      # Save the updated DataFrame to a new CSV file
      updated_blocks_info_csv_path = '/content/blocks_info.csv'
      blocks_info_df.to_csv(updated_blocks_info_csv_path, index=False)


      # Apply the labeling function to the DataFrame
      blocks_info_df['Label'] = blocks_info_df.apply(apply_labeling, axis=1)

      # Copy the content of 'Text in that block' to 'Label' for the first row
      blocks_info_df.loc[0, 'Label'] = blocks_info_df.loc[0, 'Text in that block']

      # Save the updated DataFrame to a new CSV file
      updated_blocks_info_csv_path = '/content/blocks_info.csv'
      blocks_info_df.to_csv(updated_blocks_info_csv_path, index=False)


      # Apply the replacement in the 'Label' column
      blocks_info_df['Label'] = blocks_info_df['Label'].replace({'N': 'No', 'S': 'Yes'})

      # Save the updated DataFrame to the CSV file
      updated_blocks_info_csv_path = '/content/blocks_info.csv'
      blocks_info_df.to_csv(updated_blocks_info_csv_path, index=False)

      print(f"Updated blocks info saved to {updated_blocks_info_csv_path}")


      # import pandas as pd
      # import os

      # File paths
      input_file_path = '/content/blocks_info.csv'
      output_file_path = '/content/processed_blocks_info.csv'

      # Load the DataFrame from the existing CSV file
      block_info_df = pd.read_csv(input_file_path)

      # Create an empty DataFrame with the required columns
      column_names = ['Barcode'] + [f'Block_{i}' for i in range(2, 60)]
      result_df = pd.DataFrame(columns=column_names)

      # Initialize a dictionary to hold the values
      result_row = {col: '' for col in column_names}

      # Fill the dictionary with values from 'block_info_df'
      for idx, row in block_info_df.iterrows():
          try:
              block_no = int(row['Block no'])
              label = row['Label']

              if block_no == 1:
                  result_row['Barcode'] = label
              elif 2 <= block_no <= 59:
                  result_row[f'Block_{block_no}'] = label

          except ValueError:
              # This will handle cases where the block_no is not an integer (e.g., header rows or other invalid values)
              print(f"Skipping non-numeric block number: {row['Block no']}")

      # Convert the result_row dictionary to a DataFrame
      result_row_df = pd.DataFrame([result_row])

      # Check if the output file already exists
      if os.path.exists(output_file_path):
          # Load existing data
          existing_df = pd.read_csv(output_file_path)
          # Append new row to existing data
          updated_df = pd.concat([existing_df, result_row_df], ignore_index=True)
      else:
          # If file does not exist, use the new DataFrame
          updated_df = result_row_df

      # Save the resulting DataFrame to the CSV file
      updated_df.to_csv(output_file_path, index=False)

      print(updated_df)

def pdf_to_images_and_process(pdf_path):
    # Convert PDF to images
    pages = convert_from_path(pdf_path, 300)  # 300 is the DPI (dots per inch) for high-quality images

    for i, page in enumerate(pages):
        # Save each page as an image
        image_path = f'/content/page_{i+1}.jpg'
        page.save(image_path, 'JPEG')

        # Pass the saved image to the process_img function
        process_img(image_path)

# Example usage
pdf_path = '/content/1.pdf'  # Path to your PDF file
pdf_to_images_and_process(pdf_path)


: 